In [9]:
import gct,os

In [5]:
names = [u for u in gct.list_local_graph() if u.startswith("snap_rand_N")]
len(names)

50

In [8]:
names[:2]

['snap_rand_N32', 'snap_rand_N38']

In [7]:
!pip install func_timeout

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
def run_alg(algname):
    for dataname in names:
        print("running", algname,dataname)
        retcode = os.system("timeout -k 1h 1h python run_alg.py {} {} 1".format(algname,dataname))
        if retcode !=0:
            break


In [14]:
run_alg('oslom_lpm')

running oslom_lpm snap_rand_N32
running oslom_lpm snap_rand_N38
running oslom_lpm snap_rand_N45
running oslom_lpm snap_rand_N54
running oslom_lpm snap_rand_N64
running oslom_lpm snap_rand_N76
running oslom_lpm snap_rand_N91
running oslom_lpm snap_rand_N109
running oslom_lpm snap_rand_N130
running oslom_lpm snap_rand_N156
running oslom_lpm snap_rand_N187
running oslom_lpm snap_rand_N224
running oslom_lpm snap_rand_N268
running oslom_lpm snap_rand_N321
running oslom_lpm snap_rand_N385
running oslom_lpm snap_rand_N462
running oslom_lpm snap_rand_N554
running oslom_lpm snap_rand_N664
running oslom_lpm snap_rand_N796
running oslom_lpm snap_rand_N955
running oslom_lpm snap_rand_N1146
running oslom_lpm snap_rand_N1375
running oslom_lpm snap_rand_N1650
running oslom_lpm snap_rand_N1980
running oslom_lpm snap_rand_N2376
running oslom_lpm snap_rand_N2851
running oslom_lpm snap_rand_N3421
running oslom_lpm snap_rand_N4105
running oslom_lpm snap_rand_N4926
running oslom_lpm snap_rand_N5911
running

In [21]:
import numpy as np
algs =gct.list_algorithms()
np.random.shuffle(algs)
len(algs)

72

In [ ]:
from joblib import Parallel, delayed
ret=Parallel(n_jobs=6)(delayed(run_alg)(x) for x in algs)    

In [11]:
import numpy as np
np.where(not ret), len(ret)

NameError: name 'ret' is not defined

In [84]:
results = {}
d = gct.list_all_clustering_results(print_format=False)
for u in d:
    if "snap_rand_N" in u:
        for run in d[u]:
            a = gct.load_clustering_result(u, run)
            results[u + run] = a

len(results)

2727

In [85]:
import pandas as pd
df = pd.DataFrame([(u['runname'],u['dataname'],u['timecost']) for u in results.values()], columns=['alg','datanem','timecost'])

In [86]:
missedargs = set(gct.list_algorithms()).difference(df['alg'].values)
missedargs


{'alg_Paris',
 'cdc_CliquePercolation',
 'cdc_DEMON',
 'cdc_EAGLE',
 'cdc_HDEMON',
 'igraph_community_fastgreedy',
 'igraph_community_multilevel',
 'igraph_community_optimal_modularity',
 'snap_Clauset_Newman_Moore',
 'snap_Girvan_Newman'}

In [87]:
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    display (df.sort_values("timecost"))

,alg,datanem,timecost
89,igraph_community_label_propagation,snap_rand_N38,0.000132
150,igraph_community_label_propagation,snap_rand_N45,0.000139
212,igraph_community_label_propagation,snap_rand_N54,0.000175
270,igraph_community_label_propagation,snap_rand_N64,0.000193
388,igraph_community_label_propagation,snap_rand_N91,0.000226
...,...,...,...
1985,oslom_modopt,snap_rand_N14706,3200.705344
1982,cdc_MSCD_SOM,snap_rand_N14706,3390.937801
2330,igraph_community_spinglass,snap_rand_N43909,3431.292937
2575,mcl_MCL,snap_rand_N131107,3447.448860


In [88]:
df['N']=df['datanem'].map(lambda u: int(u.split("_")[-1][1:]))

In [89]:
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    pdf=df.pivot_table(columns=['N'], index=['alg'], values='timecost')
    for x in missedargs:
        pdf=pdf.append(pd.Series(name=x))

    display(pdf)

,32,38,45,54,64,76,91,109,130,156,187,224,268,321,385,462,554,664,796,955,1146,1375,1650,1980,2376,2851,3421,4105,4926,5911,7093,8511,10213,12255,14706,17647,21176,25411,30493,36591,43909,52690,63228,75873,91047,109256,131107,157328,188793,226551
alg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
alg_GossipMap,0.598474,0.572785,0.591374,0.566338,0.589760,0.574869,0.567559,0.584993,0.609302,0.594764,0.611029,0.610482,0.615805,0.603588,0.766339,0.656093,0.675423,0.702883,0.742001,1.153000,1.196073,1.459963,1.542681,1.987415,1.969547,2.257282,2.383118,2.940774,3.843968,4.111404,5.452922,5.958948,7.135949,8.634824,8.501514,13.153406,15.752889,19.384118,21.701870,28.900443,28.393440,38.665541,47.314783,53.729638,68.070230,111.750746,100.053451,115.783500,140.284189,175.983031
alg_RelaxMap,0.016835,0.014775,0.014615,0.016021,0.016612,0.017131,0.020464,0.021847,0.024275,0.022240,0.027054,0.036976,0.040612,0.042283,0.047086,0.056380,0.066201,0.060291,0.123358,0.142762,0.203338,0.360555,0.391391,0.362439,0.575392,0.787035,1.043951,1.309919,1.208175,1.626354,1.965161,2.525559,2.984334,3.336671,4.080715,5.301832,6.435524,7.386099,7.982575,10.886286,20.527627,24.958269,28.748741,34.986918,41.921917,72.309370,63.023362,59.048001,78.427566,114.660106
alg_lso_cluster,0.014548,0.013310,0.010408,0.012483,0.012327,0.011923,0.015018,0.012094,0.012846,0.016363,0.016827,0.017408,0.021108,0.021680,0.026024,0.027418,0.033630,0.033418,0.047260,0.047184,0.059361,0.072088,0.079562,0.100528,0.132429,0.150354,0.182945,0.149633,0.198359,0.369844,0.408576,0.567309,0.770594,0.814782,0.981764,1.323651,1.742490,2.193707,2.691245,3.197136,4.078969,5.855989,7.321784,9.965339,9.554958,18.736427,15.936861,21.018839,25.351547,36.857927
alg_pg_label_propagation,0.579085,0.395923,0.402333,0.399425,0.396904,0.398895,0.396325,0.386527,0.385051,0.386762,0.394318,0.398857,0.388577,0.386668,0.401787,0.394098,0.407524,0.391850,0.393936,0.398446,0.402683,0.394658,0.396689,0.388354,0.411820,0.398774,0.404604,0.426618,0.438379,0.490653,0.518456,0.554332,0.608082,0.697155,0.744195,0.808861,0.908479,1.096859,1.255648,1.418962,2.002939,2.384418,2.898775,3.599769,4.072817,6.246617,6.162589,8.075452,8.485505,10.722002
alg_streamcom,0.014270,NaN,0.012583,NaN,0.013276,0.012935,0.013582,0.016505,NaN,0.013573,0.012788,0.012317,0.012801,0.013056,0.012544,NaN,NaN,0.014225,0.013206,0.015227,NaN,0.016476,NaN,0.019028,0.021520,0.022923,0.026234,0.029274,NaN,0.035820,0.038982,0.045193,0.051288,0.057759,NaN,0.087039,0.100481,0.118552,0.140600,0.165493,0.264655,NaN,0.392858,0.485584,0.575555,0.903849,0.878687,1.048688,1.292269,1.627761
cdc_CONGA,0.106212,0.132485,0.141127,0.151931,0.153750,0.157158,0.175614,0.184311,0.180827,0.194925,0.226002,0.211642,0.247092,0.261771,0.280643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cdc_Connected_Iterative_Scan,0.024512,0.024755,0.021325,0.028003,0.037704,0.051888,0.111423,0.160575,0.151612,0.362719,0.330983,0.757523,0.729587,1.106947,1.996415,3.609795,4.693362,6.929773,10.830857,14.034963,21.924701,36.068810,57.252887,99.470985,127.702500,221.136450,269.984748,429.845660,557.520177,528.200249,728.231022,715.010991,596.749792,583.255516,389.458877,342.725213,276.245193,322.455582,407.399040,479.677681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cdc_FastCpm,0.018798,0.017692,0.012740,0.012685,0.018970,0.014325,0.013341,0.021033,0.024101,0.025996,0.017185,0.018884,0.033583,0.024771,0.042940,0.048142,0.053690,0.067082,0.081565,0.057833,0.101557,0.127980,0.091153,0.186061,0.130744,0.227812,0.184122,0.223191,0.442721,0.422247,0.650495,0.456983,0.640848,0.680717,1.397078,1.559460,1.233330,1.491603,2.613087,3.484314,7.284742,7.087306,8.514756,11.889438,11.653016,21.509777,17.446043,24.337539,30.818426,35.011715
cdc_GCE,0.015655,0.013481,0.013502,0.013351,0.009678,0.014047,0.014713,0.016639,0.016219,0.015000,0.017362,0.020053,0.019846,0.022683,0.022385,0.029910,0.0329

In [58]:
pd.Series(name=x)

Series([], Name: snap_Girvan_Newman, dtype: float64)

In [56]:
pdf.shape

(61, 50)

In [35]:
pdf.max(axis=1)

alg
alg_RelaxMap                    114.660106
alg_lso_cluster                  36.857927
alg_pg_label_propagation          0.579085
alg_streamcom                     1.627761
cdc_CONGA                         0.280643
                                  ...     
scan_pScan                        0.597787
scan_ppScan                       0.190575
scan_ppScanSSE                    0.194713
sklearn_AffinityPropagation    2586.744437
sklearn_SpectralClustering     3053.078523
Length: 61, dtype: float64

In [36]:
os.system("ls > /tmp/a")

0

In [71]:
def run_alg(row):
    for N in row.index:
        algname=row.name
        t=row[N]
        if t>2000:
            break
        if np.isnan(t):
            dataname='snap_rand_N'+str(N)
            print("running", algname,dataname)
            if 1:
                retcode = os.system("timeout -k 1h 1h python run_alg.py {} {} 0 > /data/log/{}_{}.log 2>&1".format(algname,dataname,algname,dataname))
                if retcode !=0:
                    break


In [72]:
for i in pdf.index:
    #run_alg(pdf.loc[i])
    pass

In [73]:
args=[pdf.loc[i] for i in pdf.index]
len(args)

72

In [74]:

from joblib import Parallel, delayed
ret=Parallel(n_jobs=6)(delayed(run_alg)(x) for x in args)    

In [83]:
run_alg(pdf.loc['alg_pg_label_propagation'])

running alg_pg_label_propagation snap_rand_N38
running alg_pg_label_propagation snap_rand_N45
running alg_pg_label_propagation snap_rand_N54
running alg_pg_label_propagation snap_rand_N64
running alg_pg_label_propagation snap_rand_N76
running alg_pg_label_propagation snap_rand_N91
running alg_pg_label_propagation snap_rand_N109
running alg_pg_label_propagation snap_rand_N130
running alg_pg_label_propagation snap_rand_N156
running alg_pg_label_propagation snap_rand_N187
running alg_pg_label_propagation snap_rand_N224
running alg_pg_label_propagation snap_rand_N268
running alg_pg_label_propagation snap_rand_N321
running alg_pg_label_propagation snap_rand_N385
running alg_pg_label_propagation snap_rand_N462
running alg_pg_label_propagation snap_rand_N554
running alg_pg_label_propagation snap_rand_N664
running alg_pg_label_propagation snap_rand_N796
running alg_pg_label_propagation snap_rand_N955
running alg_pg_label_propagation snap_rand_N1146
running alg_pg_label_propagation snap_rand_N1